### RAG

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import requests

# Sua chave da API DeepSeek
DEEPSEEK_API_KEY = "XXXXXXXXXXXXXXXXXXXXX"

# Documentos base (sua base de conhecimento)
docs = [
    "O Sol é a estrela no centro do sistema solar.",
    "A Lua é o único satélite natural da Terra.",
    "O sistema solar tem oito planetas que orbitam o Sol.",
]

# Pergunta do usuário

pergunta = "Quantos planetas existem no sistema solar?"
#pergunta = "o que é o Sol?"
#pergunta = "O que é uma abobrinha"

# Recuperação do documento mais relevante via TF-IDF
vectorizer = TfidfVectorizer()
doc_vectors = vectorizer.fit_transform(docs + [pergunta])
similaridades = cosine_similarity(doc_vectors[-1], doc_vectors[:-1])
indice_relevante = similaridades.argmax()
contexto = docs[indice_relevante]

# Prompt reforçado para garantir uso exclusivo do contexto
prompt = f"""
Você é um assistente inteligente. Use apenas as informações fornecidas no contexto abaixo para responder à pergunta.

Se a resposta não estiver no contexto, responda que você não sabe.

Contexto: {docs}

Pergunta: {pergunta}

Resposta:
"""

# Chamada à API DeepSeek
url = "https://api.deepseek.com/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {DEEPSEEK_API_KEY}",
    "Content-Type": "application/json"
}
payload = {
    "model": "deepseek-chat",
    "messages": [
        {"role": "user", "content": prompt}
    ],
    "temperature": 0.3
}

response = requests.post(url, headers=headers, json=payload)

if response.ok:
    resposta = response.json()["choices"][0]["message"]["content"]
    print("Resposta da IA:", resposta)
else:
    print("Erro:", response.status_code, response.text)


Resposta da IA: De acordo com o contexto fornecido, existem oito planetas no sistema solar.


### FAISS

In [38]:
import faiss
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import requests
from sentence_transformers import SentenceTransformer

# Chave API DeepSeek
DEEPSEEK_API_KEY = "XXXXXXXXXXXXXXXXXXXXX"

# Base de documentos
docs = [
    "O Sol é a estrela no centro do sistema solar.",
    "A Lua é o único satélite natural da Terra.",
    "O sistema solar tem oito planetas que orbitam o Sol.",
]

# Usaremos embeddings com SentenceTransformer para FAISS
model_embed = SentenceTransformer('all-MiniLM-L6-v2')

# Gerar embeddings para docs
doc_embeddings = model_embed.encode(docs, convert_to_numpy=True)

# Construir índice FAISS (L2 normalizado)
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(doc_embeddings)

# Pergunta do usuário

# pergunta = "Quantos planetas existem no sistema solar?"
pergunta = "o que é o Sol?"
# pergunta = "O que é uma abobrinha"


# Gerar embedding da pergunta
query_embedding = model_embed.encode([pergunta], convert_to_numpy=True)

# Recuperar os 2 docs mais relevantes
k = 2
distances, indices = index.search(query_embedding, k)

# Juntar os docs recuperados
contexto = "\n".join([docs[i] for i in indices[0]])

# Montar prompt reforçado
prompt = f"""
Você é um assistente inteligente.

Use primeiramente **apenas** as informações fornecidas no contexto abaixo para responder à pergunta.

Se a resposta estiver no contexto, responda com base nele.

Se a resposta **não estiver no contexto**, responda usando seu próprio conhecimento, mesmo que não esteja no contexto.

Contexto: {contexto}

Pergunta: {pergunta}

Resposta:
"""

# Chamada para DeepSeek API
url = "https://api.deepseek.com/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {DEEPSEEK_API_KEY}",
    "Content-Type": "application/json"
}
payload = {
    "model": "deepseek-chat",
    "messages": [
        {"role": "user", "content": prompt}
    ],
    "temperature": 0.3
}

response = requests.post(url, headers=headers, json=payload)

if response.ok:
    resposta = response.json()["choices"][0]["message"]["content"]
    print("Resposta da IA:", resposta)
else:
    print("Erro:", response.status_code, response.text)


Resposta da IA: Com base no contexto fornecido, o Sol é a estrela no centro do sistema solar.


### Retrieval	Busca documentos relevantes
### Augmented	Aumenta o conhecimento do modelo com documentos externos
### Generation	Gera resposta com base no contexto e conhecimento próprio